In [102]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def get_product_data_la_canasta(product_html, base_url):
    sku = product_html.find("small", {"class": "sku-code"}).text.replace("SKU: ", "")
    title = product_html.find("div", {"class": "product-title"}).text.strip()
    description = product_html.find("div", {"class": "product-short-description"}).text.strip()
    quantity = product_html.find("strong").text
    price = product_html.find('span', {'class': 'total-options'})
    price = price.get("data-price")

    product_url = urljoin(base_url, product_html.find("a")["href"])

    product_details = requests.get(product_url)
    product_details.raise_for_status()  # si hay error se lanza la excepción
    detail_html = product_details.content
    details_info = BeautifulSoup(detail_html, "html.parser")
    img_url = urljoin(base_url, details_info.find("img", {"class": "lazy"})["data-src"])

    product_info = {"sku": sku, "title": title, "description": description,
                    "quantity": quantity, "price": price, "product url": product_url, "image url": img_url}
    
    return product_info

def scrape_la_canasta(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lanzar una excepción si hay un error HTTP

        soup = BeautifulSoup(response.content, "html.parser")

        categories = {}
        products = []

        menu_items = soup.find_all("li", class_="root-item menu_item_children")

        for item in menu_items:
            link = urljoin(url, str(item.find("a")["href"]))
            text = item.find("span").text
            
            # Every element of the dict has the category name as a key and the link as the value
            response = requests.get(link)
            response.raise_for_status()  # si hay error se lanza la excepción
            html = response.content
            products_info = BeautifulSoup(html, "html.parser").find_all("div", class_="row variant-fancy-box")

            for product_html in products_info:
                product_info = get_product_data_la_canasta(product_html, url)
                products.append(product_info)

            # Every element of the dict has the category name as a key and the products html as the value
            categories[text] = products_info
        df_products = pd.DataFrame(products)

    except requests.exceptions.RequestException as e:
        print(f"Error al obtener el contenido de la URL '{url}': {e}")
        # Puedes agregar aquí el manejo de errores específicos según tus necesidades
    except Exception as e:
        print(f"Error inesperado: {e}")
        # Puedes agregar aquí el manejo de errores específicos según tus necesidades
    else:
        # El código en este bloque else se ejecutará si no hay excepciones
        print("Operación exitosa. Puedes continuar con el procesamiento de 'categories'.")
        return df_products


scraped_data = scrape_la_canasta("https://supermercadoslacanasta.com/es?spcode=smlc-sogamoso")


Operación exitosa. Puedes continuar con el procesamiento de 'categories'.


In [105]:
scraped_data.to_csv("la_canasta_products.csv",index=False)